In [29]:
import cv2
import os
import pysubs2

In [30]:
def Inpainting(srcImg,x,y,xx,yy,style="text"):
    src = srcImg[x:xx,y:yy].copy()
    gray = cv2.cvtColor(src, cv2.COLOR_BGR2GRAY,1)
    if style == "title" or style == "title_number": #黑字
        thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY_INV | cv2.THRESH_OTSU)[1]
    else:
        thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)[1]
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (7, 7))
    maskImg = cv2.dilate(thresh, kernel)
    inpaintImg = cv2.inpaint(src,maskImg,3,cv2.INPAINT_NS)
    srcImg[x:xx,y:yy] = inpaintImg

In [31]:
def readAxis(filename):
    with open(filename, "r",encoding='utf-8') as f:
        for line in f.readlines():
            line = line.strip('\n')
            # print(line[0])
            line = line.split(':')
            style = line[0]
            axis[style] = list(map(int,list(line[1].split(','))))


axis = {}
readAxis("config.txt")
print(axis)

{'标题': [400, 580, 1240, 675], '羁绊标题': [535, 540, 1054, 632], '章节': [705, 505, 885, 570], '学生': [105, 900, 570, 966], '文本': [114, 988, 1500, 1154], '单选': [435, 495, 1200, 550], '选项1': [604, 435, 1029, 501], '选项2': [595, 560, 1029, 615]}


In [32]:
cap = cv2.VideoCapture("柚鸟夏 (4).mp4")
length = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
fps = cap.get(cv2.CAP_PROP_FPS)
fourcc = cv2.VideoWriter_fourcc(*'mp4v')

width = int(cap.get(3))
height = int(cap.get(4))
out = cv2.VideoWriter('new.mp4',fourcc,60,(width,height))  

frame = []
# frame = [[]] * length
for i in range(length):
    frame.append([])
subs = pysubs2.load("小夏4.ass")

for sub in subs:
    start = pysubs2.time.ms_to_frames(sub.start, fps)
    end = pysubs2.time.ms_to_frames(sub.end, fps)

    if sub.style == "文本":
        name_school = sub.text.split("：")[0]
        text = sub.text.split("：")[-1]

        if name_school != text:
            name = name_school.split('（')[0]
            school = name_school.split('（')[-1].replace("）","")
            subs.insert(0, pysubs2.SSAEvent(start=sub.start, end=sub.end, text=name+" {\\fs35 \c&H f4ca80 }"+school))
        sub.text = text

        for i in range(start,end):
            frame[i].append("学生")
            frame[i].append("文本")

    else:
        for i in range(start,end):
            frame[i].append(sub.style)
subs.save("out.ass")

In [33]:
#测试
for i in range(length):
    ret, img = cap.read()
    print(i/length*100)
    if ret:
        for style in frame[i]:
            y1 = axis[style][0]
            x1 = axis[style][1]
            y2 = axis[style][2]
            x2 = axis[style][3]
            Inpainting(img,x1,y1,x2,y2,style)
        out.write(img)
        #img = cv2.resize(img, (0, 0),fx = 0.5,fy = 0.5)
        #cv2.imshow('img',img)
    else:
        print("Error")
        break
    if cv2.waitKey(2) & 0xFF==27:
        break
cap.release()
out.release()
cv2.destroyAllWindows()

0.0
0.012776287210936502
0.025552574421873003
0.038328861632809505
0.05110514884374601
0.06388143605468251
0.07665772326561901
0.08943401047655551
0.10221029768749201
0.11498658489842851
0.12776287210936502
0.1405391593203015
0.15331544653123802
0.16609173374217454
0.17886802095311102
0.19164430816404754
0.20442059537498403
0.21719688258592054
0.22997316979685703
0.24274945700779355
0.25552574421873003
0.2683020314296665
0.281078318640603
0.29385460585153955
0.30663089306247604
0.3194071802734125
0.33218346748434907
0.34495975469528556
0.35773604190622205
0.37051232911715853
0.3832886163280951
0.39606490353903157
0.40884119074996805
0.42161747796090454
0.4343937651718411
0.44717005238277757
0.45994633959371406
0.47272262680465055
0.4854989140155871
0.4982752012265236
0.5110514884374601
0.5238277756483967
0.536604062859333
0.5493803500702695
0.562156637281206
0.5749329244921426
0.5877092117030791
0.6004854989140156
0.6132617861249521
0.6260380733358886
0.638814360546825
0.65159064775776

KeyboardInterrupt: 